<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.0/mpnn/examples/proteinmpnn_in_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ProteinMPNN in Jax!
Examples: 
- fixbb monomer design:
  - `pdb="6MRR" chains="A"`
- fixbb homooligomer design:
  - `pdb="5XZK" chains="A,B,C" homooligomer=True`
- binder design:
  - `pdb="1SSC" chains="A,B" fix_pos="A"`


In [ ]:
#@title ##Run ProteinMPNN to design new sequences for given backbone
import warnings, os
warnings.simplefilter(action='ignore', category=FutureWarning)
try:
  import colabdesign
except:
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.0")
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")

from colabdesign.mpnn import mk_mpnn_model, clear_mem
from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp

from google.colab import files

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
import re
#@markdown ### ProteinMPNN options
model_name = "v_48_020" #@param ["v_48_002", "v_48_010", "v_48_020", "v_48_030"]
#@markdown #### Input Options
pdb='6MRR' #@param {type:"string"}
pdb_path = get_pdb(pdb)
#@markdown - leave blank to get an upload prompt
chains = "A" #@param {type:"string"}
chains = re.sub("[^A-Za-z]+",",", chains)
homooligomer = False #@param {type:"boolean"}
#@markdown #### Design constraints
fix_pos = "" #@param {type:"string"}
if fix_pos == "": fix_pos = None
#@markdown - specify which positions to keep fixed in the sequence (example: `1,2-10`)
#@markdown - you can also specify chain specific constraints (example: `A1-10,B1-20`)
#@markdown - you can also specify to fix entire chain(s) (example: `A`)
inverse = False #@param {type:"boolean"}
#@markdown - inverse the `fix_pos` selection (define position to "free" [or design] instead of "fix")

rm_aa = "" #@param {type:"string"}
rm_aa = ",".join(list(re.sub("[^A-Z]+","",rm_aa.upper())))
if rm_aa == "": rm_aa = None
#@markdown - specify amino acid(s) to exclude (example: `C,A,T`)

clear_mem()
mpnn_model = mk_mpnn_model(model_name)
mpnn_model.prep_inputs(pdb_filename=pdb_path,
                       chain=chains, homooligomer=homooligomer,
                       fix_pos=fix_pos, inverse=inverse,
                       rm_aa=rm_aa)
print("lengths", mpnn_model._lengths)
if "fix_pos" in mpnn_model._inputs:
  print("the following positions will be fixed:")
  print(mpnn_model._inputs["fix_pos"])

#@markdown #### Design Options
num_seqs = 32 #@param ["0","1", "2", "4", "8", "16", "32", "64", "128", "256"] {type:"raw"}
sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
#@markdown - Sampling temperature for amino acids, T=0.0 means taking argmax, T>>1.0 means sample randomly.
parallel = True #@param {type:"boolean"}
#@markdown - sample sequences in parallel (same result, but may speedup runtime for fancy GPUs)

def compute_seqid(S):
  mask = mpnn_model._inputs["mask"].copy()
  if "fix_pos" in mpnn_model._inputs:
    mask[mpnn_model._inputs["fix_pos"]] = 0
  seq = S.argmax(-1)
  seqid = seq == mpnn_model._inputs["S"]
  seqid = (seqid * mask).sum() / mask.sum()
  return seqid

def split_seq(seq):
  if len(mpnn_model._lengths) > 1:
    return "".join(np.insert(list(seq),np.cumsum(mpnn_model._lengths[:-1]),"/"))
  else:
    return seq

if parallel:
  out = mpnn_model.sample_parallel(batch=num_seqs,
                                   temperature=sampling_temp,
                                   rescore=homooligomer)
  out["seqid"] = [compute_seqid(S) for S in out["S"]]
  for n in range(num_seqs):
    print(f'>score:{out["score"][n]:.3f}_seqid:{out["seqid"][n]:.3f}\n{split_seq(out["seq"][n])}')
else:
  outs = []
  for n in range(num_seqs):
    out = mpnn_model.sample(temperature=sampling_temp,
                            rescore=homooligomer)
    out["seqid"] = compute_seqid(out["S"])
    outs.append(out)
    print(f'>score:{out["score"]:.3f}_seqid:{out["seqid"]:.3f}\n{split_seq(out["seq"])}')
  out = jax.tree_map(lambda *x:np.stack(x,0), *outs)

In [ ]:
#@markdown ### amino acid probabilties (optional)
mode = "unconditional" #@param ["unconditional", "conditional", "conditional_fix_pos"]
#@markdown - `unconditional` - P(sequence | structure) 
#@markdown - `conditional` - P(sequence | structure, sequence)
#@markdown - `conditional_fix_pos` - P(sequence | structure, sequence[fix_pos])
show = "all" 
import plotly.express as px
from scipy.special import softmax
from colabdesign.mpnn.model import residue_constants
L = mpnn_model._len
fix_pos = mpnn_model._inputs.get("fix_pos",[])
free_pos = np.delete(np.arange(L),fix_pos)
if mode == "unconditional":
  ar_mask = np.zeros((L,L))
  logits = mpnn_model.score(ar_mask=ar_mask)["logits"]
if mode == "conditional":
  ar_mask = 1-np.eye(L)
  logits = mpnn_model.score(ar_mask=ar_mask)["logits"]
if mode == "conditional_fix_pos":
  assert "fix_pos" in mpnn_model._inputs, "no positions fixed"
  ar_mask = 1-np.eye(L)
  p = np.delete(np.arange(L),mpnn_model._inputs["fix_pos"])
  ar_mask[free_pos[:,None],free_pos[None,:]] = 0
  logits = mpnn_model.score(ar_mask=ar_mask)["logits"]

pdb_labels = np.array([f"{i}_{c}" for c,i in zip(mpnn_model.pdb["idx"]["chain"], mpnn_model.pdb["idx"]["residue"])])
pssm = softmax(logits,-1)
if show == "fix_pos":
  pdb_labels = pdb_labels[fix_pos]
  pssm = pssm[fix_pos]
if show == "free_pos":
  pdb_labels = pdb_labels[free_pos]
  pssm = pssm[free_pos]
fig = px.imshow(np.array(pssm).T,
               labels=dict(x="positions", y="amino acids", color="probability"),
               y=residue_constants.restypes + ["X"],
               #x=pdb_labels,
               zmin=0,
               zmax=1,
               template="simple_white",
              )
fig.update_xaxes(side="top")
fig.show()

In [ ]:
#@title ##Run AlphaFold Prediction on ProteinMPNN sequences
num_models = 1 #@param ["1","2","3","4","5"] {type:"raw"}
num_recycles = 1 #@param ["0","1","2","3"] {type:"raw"}
use_multimer = False #@param {type:"boolean"}
use_AF2Rank = False #@param {type:"boolean"}
#@markdown - (WIP) AF2Rank uses native structure as input template and assess the 
#@markdown agreement between sequence and structure using AlphaFold's confidence metrics.
#@markdown - AF2Rank's "composite" metric is defined as pLDDT * pTMscore. (Recommended for partial redesign of natural proteins, or interface design).
if not os.path.isdir("params"):
  os.system("mkdir params")
  os.system("curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar | tar x -C params")

if not os.path.isdir("all_pdb"):
  os.system("mkdir all_pdb")
else:
  os.system("rm all_pdb/*")

from colabdesign.af import mk_af_model
from IPython.display import HTML
clear_mem()

af_model = mk_af_model(use_multimer=use_multimer,
                       use_templates=use_AF2Rank,
                       best_metric="dgram_cce")
af_model.prep_inputs(pdb_path,
                     chains,
                     homooligomer=homooligomer)

for S in out["S"]:
  seq = S[:af_model._len,:20]
  af_model.predict(seq=seq,
                   num_recycles=num_recycles,
                   num_models=num_models,
                   verbose=False)
  rmsd = af_model.aux["log"]["rmsd"]
  ptm = af_model.aux["log"]["ptm"]
  plddt = af_model.aux["log"]["ptm"]
  if use_AF2Rank:
    af_model.aux["log"]["composite"] = ptm * plddt
    af_model._save_results(save_best=True,
                           best_metric="composite",
                           metric_higher_better=True)
  else:
    af_model._save_results(save_best=True)

  af_model._k += 1
  af_model.save_current_pdb(f"all_pdb/ptm{ptm:.3f}_plddt{plddt:.3f}_rmsd{rmsd:.3f}_n{af_model._k}.pdb")
af_model.save_pdb(f"best.pdb")

In [ ]:
#@title animate
color_by = "plddt" #@param ["chain", "plddt", "rainbow"]
HTML(af_model.animate(color_by=color_by))

In [ ]:
#@title display best protein {run: "auto"}
color = "pLDDT" #@param ["chain", "pLDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
color_HP = False #@param {type:"boolean"}
animate = False #@param {type:"boolean"}
#@markdown - if `num_models` > 1, will iterate through the models when `animate` is enabled.
af_model.plot_pdb(show_sidechains=show_sidechains,
                  show_mainchains=show_mainchains,
                  color=color, color_HP=color_HP, animate=animate)

In [ ]:
af_model.get_seq()